In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import os
import requests
import time
from random import uniform
import pandas as pd

In [3]:
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(3)
#여성 상의
#driver.get('https://shopping.naver.com/style/style/category?menu=10000532')
# 여성 하의 스커트
#driver.get('https://shopping.naver.com/style/style/category?menu=10000548')
#남성
#driver.get('https://shopping.naver.com/style/style/category?menu=10000623')
# 남성 셔츠/블라우스
driver.get('https://shopping.naver.com/style/style/category?menu=10000631')

In [4]:
# 스크롤
link_list = []

for scr in range(10):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.select('div._1aNmedx3I0 > ul._3KH4iCkMJ_ > li > div > a')
    f = uniform(2.0, 2.5)
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(f)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(f)
    
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    for link in links:
        link_list.append('https://shopping.naver.com'+link['href'])
        
    
    if new_height == last_height:

            # try again (can be removed)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(f)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")

            # check if the page height has remained the same
            if new_height == last_height:
                # if so, you are done
                break
            # if not, move on to the next loop
            else:
                last_height = new_height
                continue

In [5]:
link_list

['https://shopping.naver.com/style/style/stores/100384526/products/4861380965',
 'https://shopping.naver.com/style/style/stores/1000010948/products/4861372509',
 'https://shopping.naver.com/style/style/stores/1000010948/products/4861361457',
 'https://shopping.naver.com/style/style/stores/100343274/products/4861342461',
 'https://shopping.naver.com/style/style/stores/100228692/products/4861314148',
 'https://shopping.naver.com/style/style/stores/100343863/products/4851249539#INTRODUCE',
 'https://shopping.naver.com/style/style/stores/100228692/products/4861270967',
 'https://shopping.naver.com/style/style/stores/100498593/products/4861269438',
 'https://shopping.naver.com/style/style/stores/100364920/products/4861247398',
 'https://shopping.naver.com/style/style/stores/100228692/products/4861196290',
 'https://shopping.naver.com/style/style/stores/1000004719/products/4861164666',
 'https://shopping.naver.com/style/style/stores/100066980/products/4218952039#INTRODUCE',
 'https://shoppin

In [6]:
link_list = list(set(link_list))

In [7]:
# csv 파일로 저장
import pandas as pd
results = pd.DataFrame(link_list, columns=['link_addr'])
results.to_csv('./link_addr_M_b$s.csv', header=False) #, mode='a', header=False)

In [8]:
#csv 파일 가져오기
import csv
f = open('./link_addr_M_b$s.csv','rt')
link_addr_file = csv.reader(f)

In [9]:
link_list = []
for row in link_addr_file :
    link_list.append(row[1])

In [10]:
#상세 페이지 들어가서 원하는 정보 저장
i = 0

while i < len(link_list):
    text_list = []
    
    driver.get(link_list[i])
    print(link_list[i])
    
    f = uniform(1.5, 1.7)
    time.sleep(f)
    
    a = ""
    a = link_list[i].split("/products/")
    a[1][:10]
    
    i += 1
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        images = soup.find('div','_396uGDX0Fi')
        # soup.select -> soup.find
        # ('div._3utq7rRVBL > div.editor_wrap > div > div > div.se_component_wrap > div.se_component > 
        # div.se_sectionArea > div.se_editArea > div.se_viewArea > a > img')
        name = soup.find('h3','_2IA5sp7BRM').get_text() # 동일
        price = soup.find('strong','WvptIJtvXo').get_text()  
        tag=soup.find('div','_24FKuaBdoK').get_text() # 동일
        info = [soup.find_all('th','_15qeGNn6Dt')[n].string for n in range(0,20)]  # 동일
        detail = [soup.find_all('td','jvlKiI0U_y')[n].string for n in range(0,20)] # 동일
        texts = soup.find_all('p', 'se_textarea') #동일


        if not os.path.isdir(a[1][:10]):
            os.mkdir(a[1][:10])     

        for text in texts:
            text_list.append(text.get_text().strip().replace(u'\xa0', u' '))
            text_list2 = [zz for zz in text_list if zz]

        # 긁어올 카테고리 명을 입력하세요     
        data_1 = ({'category':'남성/하의/셔츠_남방', 'name': [name], 'price': [price] , 'tag':[tag], 'size':[text_list2]})   #, 'price': [price]
        df1 = pd.DataFrame(data_1)

        data_2 = [detail]
        df2 = pd.DataFrame(data_2, columns = info)

        df3 = pd.concat([df1,df2], axis=1)

        df3.to_csv(".\\"+a[1][:10]+"\\csv_"+a[1][:10]+".csv", index = True, encoding='utf-8')

        for image in images:

            src_page = requests.get(image['src'], headers={"Referer":"https://www.naver.com/"})
            src_image = src_page.content
            with open(".\\"+a[1][:10]+"\\img_"+a[1][:10]+'_'+str(images.index(image))+".png", "wb") as downfile:
                downfile.write(src_image)
            f = uniform(0.4, 0.6)    
            time.sleep(f)

    except:
        print("상품정보 가져오기 오류발생")
        pass

https://shopping.naver.com/style/style/stores/100410095/products/4856334602
https://shopping.naver.com/style/style/stores/100025403/products/4860432524
https://shopping.naver.com/style/style/stores/1000016359/products/4859082080
https://shopping.naver.com/style/style/stores/100645355/products/4852000755
https://shopping.naver.com/style/style/stores/1000013972/products/4846859600#INTRODUCE
https://shopping.naver.com/style/style/stores/1000009970/products/4856944743
https://shopping.naver.com/style/style/stores/100138564/products/4860325645
https://shopping.naver.com/style/style/stores/100095832/products/4851020215#INTRODUCE
https://shopping.naver.com/style/style/stores/1000014785/products/4856060804
https://shopping.naver.com/style/style/stores/1000013496/products/4858622501
https://shopping.naver.com/style/style/stores/100022886/products/4860474459
https://shopping.naver.com/style/style/stores/1000018012/products/4858927245
https://shopping.naver.com/style/style/stores/1000018012/produ

https://shopping.naver.com/style/style/stores/100228692/products/4861314148
https://shopping.naver.com/style/style/stores/100501068/products/4858858530
https://shopping.naver.com/style/style/stores/1000013972/products/4848591346#INTRODUCE
https://shopping.naver.com/style/style/stores/100060070/products/4860982105
https://shopping.naver.com/style/style/stores/100032385/products/4856452403
https://shopping.naver.com/style/style/stores/100239170/products/4859206305#INTRODUCE
https://shopping.naver.com/style/style/stores/100228692/products/4861196290
https://shopping.naver.com/style/style/stores/1000017143/products/4856487082
https://shopping.naver.com/style/style/stores/1000018012/products/4858048888
https://shopping.naver.com/style/style/stores/100095832/products/4850977529#INTRODUCE
https://shopping.naver.com/style/style/stores/100025403/products/4855895945
https://shopping.naver.com/style/style/stores/1000018568/products/4859101628
https://shopping.naver.com/style/style/stores/10013689

https://shopping.naver.com/style/style/stores/100343863/products/4845316197#INTRODUCE
https://shopping.naver.com/style/style/stores/100358546/products/4856526855
https://shopping.naver.com/style/style/stores/100358546/products/4857784427
https://shopping.naver.com/style/style/stores/100239170/products/4854258581#INTRODUCE
https://shopping.naver.com/style/style/stores/1000013957/products/4859037234
https://shopping.naver.com/style/style/stores/100410511/products/4858519366
https://shopping.naver.com/style/style/stores/100679856/products/4860590588
https://shopping.naver.com/style/style/stores/100343274/products/4860150797
https://shopping.naver.com/style/style/stores/1000021603/products/4857124690
https://shopping.naver.com/style/style/stores/1000021603/products/4858697452
https://shopping.naver.com/style/style/stores/1000016196/products/4856577469


In [11]:
link_list

['https://shopping.naver.com/style/style/stores/100410095/products/4856334602',
 'https://shopping.naver.com/style/style/stores/100025403/products/4860432524',
 'https://shopping.naver.com/style/style/stores/1000016359/products/4859082080',
 'https://shopping.naver.com/style/style/stores/100645355/products/4852000755',
 'https://shopping.naver.com/style/style/stores/1000013972/products/4846859600#INTRODUCE',
 'https://shopping.naver.com/style/style/stores/1000009970/products/4856944743',
 'https://shopping.naver.com/style/style/stores/100138564/products/4860325645',
 'https://shopping.naver.com/style/style/stores/100095832/products/4851020215#INTRODUCE',
 'https://shopping.naver.com/style/style/stores/1000014785/products/4856060804',
 'https://shopping.naver.com/style/style/stores/1000013496/products/4858622501',
 'https://shopping.naver.com/style/style/stores/100022886/products/4860474459',
 'https://shopping.naver.com/style/style/stores/1000018012/products/4858927245',
 'https://shop